In [1]:
!pip install scikit-learn tensorflow

In [2]:
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import fashion_mnist
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from random import uniform, randint, sample
from abc import ABC, abstractmethod
from copy import deepcopy


In [3]:
class Individuo:
    def __init__(self, cromossomo):
        self.cromossomo = cromossomo
        self.acuracia = None
        self.loss = None

    def __str__(self):
        return ('Cromossomo: {} -> Acuracia: {}%, Loss: {}%'.
                format(self.cromossomo, self.acuracia, self.loss))


In [4]:
class Algoritmo(ABC):
    def __init__(self):
        self.min = self.definirMinimo()
        self.max = self.definirMaximo()
        self.qtd_genes = len(self.max)
        self.indices_float = self.definirIndices()

    @abstractmethod
    def __str__(self):
        pass

    @abstractmethod
    def definirMinimo(self):
        pass

    @abstractmethod
    def definirMaximo(self):
        pass

    @abstractmethod
    def gerarModelo(self, pos):
        pass

    def definirIndices(self):
        indices = []

        for i in range(self.qtd_genes):
            if not isinstance(self.min[i], int):
                indices.append(i)

        return indices

    def gerarCromossomo(self):
        cromossomo = []

        for i in range(self.qtd_genes):
            if i in self.indices_float:
                cromossomo.append(round(uniform(self.min[i], self.max[i]), 4))
            else:
                cromossomo.append(randint(self.min[i], self.max[i]))

        return cromossomo

    def gerarGene(self, i):
        if i in self.indices_float:
            return round(uniform(self.min[i], self.max[i]), 4)

        return randint(self.min[i], self.max[i])


In [5]:
class MLP(Algoritmo):
    def __str__(self):
        return "MULTI LAYER PERCEPTRON"

    def definirMinimo(self):
        return [50, 0.0001, 0.0001, 0.5, 100]

    def definirMaximo(self):
        return [300, 0.1, 0.01, 0.99, 400]

    def gerarModelo(self, genes):
        return MLPClassifier(
            hidden_layer_sizes=(genes[0],),
            learning_rate_init=genes[1],
            alpha=genes[2],
            momentum=genes[3],
            max_iter=genes[4],
            solver="sgd",
            activation="relu",
            n_iter_no_change=20,
            early_stopping=False,
            verbose=False,
        )


In [6]:
def definirAlgoritmo(tipo_algoritmo):
    if tipo_algoritmo == "MLPClassifier":
        return MLP()

    return None


def selecionarPorTorneio(populacao):
    pai1, pai2, mae1, mae2 = sample(populacao, 4)

    if pai1.f1 > pai2.f1:
        pai = pai1
    else:
        pai = pai2

    if mae1.f1 > mae2.f1:
        mae = mae1
    else:
        mae = mae2

    return deepcopy(pai), deepcopy(mae)


In [13]:
class GA:
    def __init__(self, num_individuos, tipo_algoritmo):
        self.num_individuos = num_individuos
        self.algoritmo = definirAlgoritmo(tipo_algoritmo)
        self.qtd_genes = self.algoritmo.qtd_genes
        self.chance_de_mutar = 1 / self.qtd_genes

        self.x_treinamento = None
        self.y_treinamento = None
        self.x_teste = None
        self.y_teste = None

    def definirXY(self):
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
        x_train = x_train.reshape((x_train.shape[0], -1)).astype("float32")
        x_test = x_test.reshape((x_test.shape[0], -1)).astype("float32")

        scaler = MinMaxScaler()

        self.x_treinamento = scaler.fit_transform(x_train)
        self.y_treinamento = y_train
        self.x_teste = scaler.transform(x_test)
        self.y_teste = y_test

    def gerarPopulacao(self):
        populacao = []

        for _ in range(self.num_individuos):
            populacao.append(Individuo(self.algoritmo.gerarCromossomo()))

        return populacao

    def ajustarMetricas(self, cromossomo):
        modelo = self.algoritmo.gerarModelo(cromossomo)
        modelo.fit(self.x_treinamento, self.y_treinamento)
        previsoes = modelo.predict(self.x_teste)

        acuracia = round(accuracy_score(self.y_teste, previsoes) * 100, 1)
        loss = round(modelo.loss_, 4)

        return acuracia, loss

    def executar(self):
        populacao = self.gerarPopulacao()

        with open(str(self.algoritmo) + ".txt", "w") as file:
            for individuo in populacao:
                individuo.acuracia, individuo.loss = self.ajustarMetricas(individuo.cromossomo)
                file.write(str(individuo) + '\n')


In [14]:
ga = GA(5, 'MLPClassifier')
ga.definirXY()
ga.executar()


C:\Users\walter.cavalcante\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (282) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\walter.cavalcante\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (230) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\walter.cavalcante\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (107) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\walter.cavalcante\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimize

AttributeError: 'MLPClassifier' object has no attribute 'loss_'